CS4001/4042 Assignment 1, Part B, Q4
---

Model degradation is a common issue faced when deploying machine learning models (including neural networks) in the real world. New data points could exhibit a different pattern from older data points due to factors such as changes in government policy or market sentiments. For instance, housing prices in Singapore have been increasing and the Singapore government has introduced 3 rounds of cooling measures over the past years (16 December 2021, 30 September 2022, 27 April 2023).

In such situations, the distribution of the new data points could differ from the original data distribution which the models were trained on. Recall that machine learning models often work with the assumption that the test distribution should be similar to train distribution. When this assumption is violated, model performance will be adversely impacted.  In the last part of this assignment, we will investigate to what extent model degradation has occurred.




---



---



Your co-investigators used a linear regression model to rapidly test out several combinations of train/test splits and shared with you their findings in a brief report attached in Appendix A below. You wish to investigate whether your deep learning model corroborates with their findings.

In [1]:
#!pip install alibi-detect

In [2]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from alibi_detect.cd import TabularDrift

> Evaluate your model from B1 on data from year 2022 and report the test R2.

In [3]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

In [4]:
df = pd.read_csv('hdb_price_prediction.csv')

# TODO: Enter your code here

# Training Data Set: Year 2019 and before
df_train = df[df['year'] <= 2019].copy()
# Validation Data Set: Year 2020
df_val = df[df['year'] == 2020].copy()
# Testing Data Set: Year 2022
df_test = df[df['year'] == 2022].copy()

# Dropping Unncessary Columns
df_train.drop(columns=['year','full_address'], inplace=True)
df_val.drop(columns=['year','full_address'], inplace=True)
df_test.drop(columns=['year','full_address'], inplace=True)

print("Training Data (2019):", df_train.shape)
print("Testing Data (2022):", df_test.shape)

Training Data (2019): (64057, 12)
Testing Data (2022): (26702, 12)


In [5]:
num_col_names = ['dist_to_nearest_stn','dist_to_dhoby','degree_centrality','eigenvector_centrality',
                 'remaining_lease_years','floor_area_sqm']
cat_col_names = ['month','town','flat_model_type','storey_range']

In [6]:
data_config = DataConfig(
    target=["resale_price"],  
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
)
trainer_config = TrainerConfig(
    auto_lr_find=True,  # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=50,
)
optimizer_config = OptimizerConfig()

model_config = CategoryEmbeddingModelConfig(
    task="regression",
    layers="50",  
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

2023-10-13 19:00:58,565 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off


In [7]:
from torch_optimizer import QHAdam
# Training Tabular Model
tabular_model.fit(df_train, 
                  validation=df_val, 
                  optimizer=QHAdam)

Global seed set to 42
2023-10-13 19:00:58,592 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-10-13 19:00:58,597 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
2023-10-13 19:00:58,655 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_tabular/models/base_model.py:147: UserWarning: Plotly is not installed. Please install plotly to log logits. You can install plotly using pip install plotly or install PyTorch Tabular using pip install pytorch-tabular[all]
  warnings.warn(
2023-10-13 19:00:58,672 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be remov

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.5754399373371567
Restoring states from the checkpoint path at /Users/enlih/Library/CloudStorage/OneDrive-NanyangTechnologicalUniversity/SC4001 Neural Network & Deep Learning/Project/Project 1 Submission/.lr_find_59755f75-a0da-40bc-869d-a2aafd420b33.ckpt
Restored all states from the checkpoint file at /Users/enlih/Library/CloudStorage/OneDrive-NanyangTechnologicalUniv

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  2.9 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.5 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 4.5 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.5 K                                                                                                
Total estimated model params size (MB): 0

Output()

2023-10-13 19:01:16,685 - {pytorch_tabular.tabular_model:584} - INFO - Training the model completed
2023-10-13 19:01:16,685 - {pytorch_tabular.tabular_model:1258} - INFO - Loading the best model
/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_lightning/utilities/cloud_io.py:33: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.get_filesystem` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.cloud_io.get_filesystem` instead.
  rank_zero_deprecation(


In [8]:
evaluation = tabular_model.evaluate(df_test)
predicted = tabular_model.predict(df_test)

Output()

/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You 
called `self.log('test_loss', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You 
called `self.log('test_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by
doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(

/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       16267036672.0       │
│  test_mean_squared_error  │       16267036672.0       │
└───────────────────────────┴───────────────────────────┘

Output()

In [9]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Getting the True Values and Predicted Values for RMSE and R2
y_true = predicted['resale_price']
y_pred = predicted['resale_price_prediction']

r2 = r2_score(y_true, y_pred)
print("R^2 Score for 2022:", r2)
mse = mean_squared_error(y_true, y_pred)
print("Root Mean Squared Error (RMSE) for 2022:", np.sqrt(mse))

R^2 Score for 2022: 0.43884727141390145
Root Mean Squared Error (RMSE) for 2022: 127542.29726175782


> Evaluate your model from B1 on data from year 2023 and report the test R2.

In [10]:
# TODO: Enter your code here

# Testing Data Set: Year 2023
df_test_2 = df[df['year'] == 2023].copy()
df_test_2.drop(columns=['year','full_address','nearest_stn'], inplace=True)

print("Training Data (2019):", df_train.shape)
print("Testing Data (2023):", df_test_2.shape)

Training Data (2019): (64057, 12)
Testing Data (2023): (16424, 11)


In [11]:
evaluation_2 = tabular_model.evaluate(df_test_2)
predicted_2 = tabular_model.predict(df_test_2)

Output()

/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You 
called `self.log('test_loss', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You 
called `self.log('test_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by
doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(

/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       24701362176.0       │
│  test_mean_squared_error  │       24701362176.0       │
└───────────────────────────┴───────────────────────────┘

Output()

In [12]:
# Getting the True Values and Predicted Values for RMSE and R2
y_true_2 = predicted_2['resale_price']
y_pred_2 = predicted_2['resale_price_prediction']

r2 = r2_score(y_true_2, y_pred_2)
print("R^2 Score for 2023:", r2)
mse = mean_squared_error(y_true_2, y_pred_2)
print("Root Mean Squared Error (RMSE) for 2023:", np.sqrt(mse))

R^2 Score for 2023: 0.1621259354739173
Root Mean Squared Error (RMSE) for 2023: 157166.67527449684


> Did model degradation occur for the deep learning model?


\# TODO: \< 
> Yes Model Degradation occurred, the R^2 value when the model was **tested on 2021 in B1 was 0.777** but it **dropped rapidly to 0.439 for 2022** then to **0.162 for 2023**.
\>



---



---



Model degradation could be caused by [various data distribution shifts](https://huyenchip.com/2022/02/07/data-distribution-shifts-and-monitoring.html#data-shift-types): covariate shift (features), label shift and/or concept drift (altered relationship between features and labels).
There are various conflicting terminologies in the [literature](https://www.sciencedirect.com/science/article/pii/S0950705122002854#tbl1). Let’s stick to this reference for this assignment.

> Using the **Alibi Detect** library, apply the **TabularDrift** function with the training data (year 2019 and before) used as the reference and **detect which features have drifted** in the 2023 test dataset. Before running the statistical tests, ensure you **sample 1000 data points** each from the train and test data. Do not use the whole train/test data. (Hint: use this example as a guide https://docs.seldon.io/projects/alibi-detect/en/stable/examples/cd_chi2ks_adult.html)


In [13]:
# Dropping Resale Price since its the target
train_copy = df_train.copy()
train_copy.drop(columns=['resale_price','month'],inplace=True)

test_copy = df_test.copy()
test_copy.drop(columns=['resale_price','month'],inplace=True)

feature_names = train_copy.columns
feature_names

Index(['town', 'nearest_stn', 'dist_to_nearest_stn', 'dist_to_dhoby',
       'degree_centrality', 'eigenvector_centrality', 'flat_model_type',
       'remaining_lease_years', 'floor_area_sqm', 'storey_range'],
      dtype='object')

In [14]:
# TODO: Enter your code here
sample_train = train_copy.sample(1000, random_state = 42)
sample_test = test_copy.sample(1000, random_state = 42)

categories_per_feature = {f: None for f in range(sample_train.values.shape[1])}
cd = TabularDrift(sample_train.values, 
                  p_val=.05, 
                  categories_per_feature=categories_per_feature)
preds = cd.predict(sample_test.values)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))

Drift? Yes!


In [15]:
fpreds = cd.predict(sample_test.values, drift_type='feature')
for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = feature_names[f]
    is_drift = fpreds['data']['is_drift'][f]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

town -- Drift? Yes! -- Chi2 44.147 -- p-value 0.010
nearest_stn -- Drift? Yes! -- Chi2 100.058 -- p-value 0.040
dist_to_nearest_stn -- Drift? No! -- Chi2 1787.333 -- p-value 0.251
dist_to_dhoby -- Drift? No! -- Chi2 1787.333 -- p-value 0.251
degree_centrality -- Drift? No! -- Chi2 3.300 -- p-value 0.348
eigenvector_centrality -- Drift? Yes! -- Chi2 100.058 -- p-value 0.040
flat_model_type -- Drift? Yes! -- Chi2 66.537 -- p-value 0.000
remaining_lease_years -- Drift? Yes! -- Chi2 833.791 -- p-value 0.000
floor_area_sqm -- Drift? Yes! -- Chi2 154.319 -- p-value 0.007
storey_range -- Drift? No! -- Chi2 19.503 -- p-value 0.108


> Assuming that the flurry of housing measures have made an impact on the relationship between all the features and resale_price (i.e. P(Y|X) changes), which type of data distribution shift possibly led to model degradation?


\# TODO: \<Enter your answer here\>

> From your analysis via TabularDrift, which features contribute to this shift?


\# TODO: \<
1. In the context of the analysis, theres is p-value shown together with the drift result. For the features that have shown to be drifted, their p-value is < 0.05 (Significance Level), we will reject the null hypothesis (no change or drift in the feature's distribution) and conclude there is significant evidence of data drift in the feature.

2. The features are "town", "nearest_stn", "eigenvector_centrality", "flat_model_type", "remaining_lease_years", "floor_area_sqm".
\>

> Suggest 1 way to address model degradation and implement it, showing improved test R2 for year 2023.


 \# TODO: \< 
We should be training data that is closer to the year of testing data and validation data, in my case i will select:
1. training data to be before and inclusive of **year 2021**
2. validation data to be in **year 2022** 
3. testing data to be in **year 2023**
\>

In [16]:
# TODO: Enter your code here

# The Choice of the 2021 and before is according to Appendix A
final_train = df[(df['year'] <= 2021)]
final_val = df[df['year'] == 2022]
final_test = df[df['year'] == 2023]

data_config = DataConfig(
    target=["resale_price"],  
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
)
trainer_config = TrainerConfig(
    auto_lr_find=True,  # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=50,
)
optimizer_config = OptimizerConfig()

model_config = CategoryEmbeddingModelConfig(
    task="regression",
    layers="50",  
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

2023-10-13 19:01:18,676 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off


In [17]:
# Training Tabular Model
tabular_model.fit(final_train, 
                  validation=final_val, 
                  optimizer=QHAdam)

Global seed set to 42
2023-10-13 19:01:18,691 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-10-13 19:01:18,699 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
2023-10-13 19:01:18,794 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_tabular/models/base_model.py:147: UserWarning: Plotly is not installed. Please install plotly to log logits. You can install plotly using pip install plotly or install PyTorch Tabular using pip install pytorch-tabular[all]
  warnings.warn(
2023-10-13 19:01:18,812 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be remov

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.5754399373371567
Restoring states from the checkpoint path at /Users/enlih/Library/CloudStorage/OneDrive-NanyangTechnologicalUniversity/SC4001 Neural Network & Deep Learning/Project/Project 1 Submission/.lr_find_d13b5d72-f79a-4303-bae1-0c7212754f66.ckpt
Restored all states from the checkpoint file at /Users/enlih/Library/CloudStorage/OneDrive-NanyangTechnologicalUniv

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  3.0 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.6 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 4.6 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.6 K                                                                                                
Total estimated model params size (MB): 0

Output()

2023-10-13 19:01:56,660 - {pytorch_tabular.tabular_model:584} - INFO - Training the model completed
2023-10-13 19:01:56,660 - {pytorch_tabular.tabular_model:1258} - INFO - Loading the best model
/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_lightning/utilities/cloud_io.py:33: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.get_filesystem` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.cloud_io.get_filesystem` instead.
  rank_zero_deprecation(


In [18]:
final_evaluation = tabular_model.evaluate(final_test)
final_predicted = tabular_model.predict(final_test)

Output()

/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You 
called `self.log('test_loss', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You 
called `self.log('test_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by
doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(

/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       17967239168.0       │
│  test_mean_squared_error  │       17967239168.0       │
└───────────────────────────┴───────────────────────────┘

Output()

In [19]:
# Getting the True Values and Predicted Values for RMSE and R2
y_true_final = final_predicted['resale_price']
y_pred_final = final_predicted['resale_price_prediction']

r2 = r2_score(y_true_final, y_pred_final)
print("R^2 Score for 2023:", r2)
mse = mean_squared_error(y_true_final, y_pred_final)
print("Root Mean Squared Error (RMSE) for 2023:", np.sqrt(mse))

R^2 Score for 2023: 0.39054845283895967
Root Mean Squared Error (RMSE) for 2023: 134041.9336839462


There is an improvement of R^2 score for the year 2023 showing an increase from **0.162** to **0.391** when the testing and validation data was modified accordingly.

### Appendix A



Here are our results from a linear regression model. We used StandardScaler for continuous variables and OneHotEncoder for categorical variables.

While 2021 data can be predicted well, test R2 dropped rapidly for 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| Year <= 2020 | 2021     | 0.76    |
| Year <= 2020 | **2022**     | 0.41    |
| Year <= 2020 | **2023**     | **0.10**   |



Similarly, a model trained on 2017 data can predict 2018-2021 well (with slight degradation in performance for 2021), but drops drastically in 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2017         | 2018     | 0.90    |
|              | 2019     | 0.89    |
|              | 2020     | 0.87    |
|              | 2021     | 0.72    |
|              | **2022**     | **0.37**    |
|              | **2023**     | **0.09**    |

With the test set fixed at year 2021, training on data from 2017-2020 still works well on the test data, with minimal degradation. Training sets closer to year 2021 generally do better.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2020         | 2021     | 0.81    |
| 2019         | 2021     | 0.75    |
| 2018         | 2021     | 0.73    |
| 2017         | 2021     | 0.72    |